# 최대값의 위치를 구하는 함수 작성

In [6]:
list = [1,2,3,4,4,6,123,4,5,645,3]
def SearchIndex(data_list):
    return data_list.index(max(data_list))
SearchIndex(list)

9

# random

In [2]:
import random
import time

for x in range(10):
    time.sleep(random.random())
    print(x)

0
1
2
3
4
5
6
7
8
9


In [13]:
random.uniform(1, 200) # 설정한 범위 안에서 랜덤 값 출력

188.36346926454485

In [15]:
random.randint(1, 10) # 설장한 범위 안에서 랜덤 정수 값 출력

8

# pickle
* 0과 1로 저장된 파일(디코딩 없이 사람이 읽을 수 없음)

In [3]:
import pickle
with open("./leader.pkl", "wb") as f:
    pickle.dump([random.choice(["가","나","다"]) for x in range(100)], f)
with open("./leader.pkl", "rb") as f:
    lead = pickle.load(f)
    
count_dict = dict()
for x in lead:
    if x in count_dict:
        count_dict[x] += 1
    else: count_dict[x] = 1
    
#from operator import itemgetter # operator 아래 itemgetter import
import operator
print(sorted(count_dict.items(), key=operator.itemgetter(1))) # dict 정렬

[('나', 28), ('다', 31), ('가', 41)]


# 폴더의 파일 리스트

In [5]:
import os
os.getcwd()
os.mkdir("./melon")

* 가수명 폴더 생성
* 네이밍룰 가수-제목.txt

In [184]:
import requests
from bs4 import BeautifulSoup
head = {
    'user-agent': 
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36 Edg/92.0.902.84"
}
target_url = "https://www.melon.com/chart/index.htm"
r = requests.get(target_url, headers=head)

In [185]:
r.status_code

200

In [186]:
bs = BeautifulSoup(r.text)
bs_rt = bs.findAll("span", class_ = "checkEllipsis") # 폴더 생성부터 실습한 후 가사 수집을 하려 했으나 접근 계획이 잘못됨

In [187]:
bs_rt[0]

<span class="checkEllipsis" style="display:none"><a href="javascript:melon.link.goArtistDetail('2743730');" title="The Kid LAROI - 페이지 이동">The Kid LAROI</a>, <a href="javascript:melon.link.goArtistDetail('420621');" title="Justin Bieber - 페이지 이동">Justin Bieber</a></span>

In [190]:
bs_rt2 = bs.findAll("a", class_ = "btn button_icons type03 song_info")
print(bs_rt2[0])

<a class="btn button_icons type03 song_info" href="javascript:melon.link.goSongDetail('33658563');" title="STAY 곡정보"><span class="none">곡정보</span></a>


In [189]:
# 폴더 생성 로직
str = ""
count = 0
for x in bs_rt:
    y = x.findChildren("a")
    for data in y:
        if count != 0: str += ", "
        str += data['title'][0:data['title'].find(' -')]
        count += 1
    if not os.path.isdir("./melon/" + str):
        os.mkdir('./melon/' + str)
    count = 0
    str = ""
    


In [2]:
########
# import 파이썬에서 제공되는 기능을 사용하기 위해 선언
# 가독성을 높이기 위해 상단에 모아서 선언
import os
import requests
import re
from bs4 import BeautifulSoup
########

# 현재 폴더에 melon 폴더가 있는지 확인 후 없으면 생성
# 폴더가 존재하는데 생성을 시도하는 경우 오류 발생
if not os.path.isdir("./melon"): os.mkdir("./melon")

# 웹 페이지 접근 시, head 부분에 사용자의 정보가 기록되어 서버에 전송
# 파이썬으로 접근 시, 데이터 조회를 막는 경우를 우회하기 위해 일반 사용자 브라우저인 것처럼 서버에 전달
head = {
    'user-agent': 
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36 Edg/92.0.902.84"
}
target_url = "https://www.melon.com/chart/index.htm" # 브라우저 주소창에 보이는 멜론 차트 실제 주소

r = requests.get(target_url, headers=head) # 위에 만든 head와 url을 활용해 접속 후 데이터를 r에 저장
bs = BeautifulSoup(r.text) # BeautifulSoup을 통해 데이터에 접근하기 쉽게 변형

# <a class = "btn button_icons type03 song_info".. a 태그와 해당 클래스 명을 가진 모든 요소를 반환
# 크롬 개발자 도구(f12)를 통해 확인 가능
bs_rt = bs.findAll("a", class_ = "btn button_icons type03 song_info")


for x in bs_rt:
    # 곡 상세정보에 접근
    # 방식은 위와 동일
    # https://www.melon.com/song/detail.htm?songId=33658563
    r2 = requests.get("https://www.melon.com/song/detail.htm?songId=" + re.findall("\d+", x['href'])[0], headers=head)
    bs2 = BeautifulSoup(r2.text)
    bs_rt2 = bs2.findAll("a", class_ = "artist_name")
    
    # 폴더 생성 로직

    # ", "를 찍을 위치를 구분하기 위해 count 변수 선언
    # 아티스트는 2명 이상이 될 수 있고, 첫번째 아티스트의 이름 앞에는 ", "를 출력하면 안되기 때문에 사용
    count = 0 
    str = "" # 이름 저장용 변수
    for a_name in bs_rt2:
        # 조회된 데이터에 빈 값(None)이 포함되어 조건문을 통해 None이 아닌 경우에만 이후 로직 실행
        if a_name.find("span") is not None:
            if count != 0: str += ", " # count가 0이 아닌 경우(첫번째가 아닌 경우)에만 ", "를 이름 앞에 출력
            str += a_name.find("span").text # str에 이름 저장
            
            # 실행 후 count 값 1 증가(아티스트가 3명인 경우 로직 완료후 count 값은 3, 뒤가 아닌 앞에 ", "를 붙이기 때문에 마지막 아티스트는 고려하지 않음)
            # 해당 반복문이 종료되는 시점에는 count의 값이 0보다 크지만, 첫번째 곡을 저장한 뒤 제일 처음 반복문부터 다시 시작하기 때문에 count = 0이 다시 선언되므로 값이 0으로 초기화
            count += 1
    
    # 위 로직의 실행으로 STAY의 아티스트 명은 str에 "The Kid LAROI, Justin Bieber"로 저장되어 있는 상태
    if not os.path.isdir("./melon/" + str): # melon 폴더 아래 str을 합친 디렉토리(./melon/The Kid LAROI, Justin Bieber)가 있는지 확인 후 없는 경우 생성
        os.mkdir('./melon/' + str)
        
    # 가사 파일 생성 로직
    bs_rt2 = bs2.find("div", class_ = "song_name")
    song_name = bs_rt2.text.replace("곡명", "").strip()
    
    # 가사 저장
    bs_rt2 = bs2.find("div", class_ = "lyric")
#     content = bs_rt2.text.strip() # 개행 문자가 제거되어 가사 변형 발생
#     content = bs_rt2.get_text(separator='|br|', strip=True).split('|br|') # 배열로 반환되어 추가 작업이 필요
    content = bs_rt2.get_text(separator='|br|', strip=True).replace('|br|', " ")
#     content = BeautifulSoup(str(bs_rt2).replace("<br/>", "\n")).text.strip() # 강사님 로직 수정 적용 필요

    
    with open("./melon/" + str + "/" + song_name + ".txt", "w", encoding="utf-8") as f: # 파일 생성
        f.write(content)

In [2]:
# help(BeautifulSoup.findAll)
type("1")

str

# 자연어 처리

In [2]:
from konlpy.tag import Komoran

In [3]:
komoran = Komoran()
komoran.nouns("안녕하세요")

['안녕하세요']

In [42]:
with open("./melon/10CM/10CM-고백.txt", "r") as f:
    ly = f.read()
    
# NNP, NNG
list_count = dict()
for x in komoran.pos(ly):
    if x[1] == "NNP" or x[1] == "NNG":
        print(x[0], x[1])

때 NNG
첫사랑 NNP
진행 NNG
말 NNG
2년 NNP
전 NNP
게 NNG
기분 NNG
아냐 NNP
가슴 NNG
생각 NNG
손 NNG
때 NNG
사람 NNG
상처 NNP
날 NNG
모두 NNG
때 NNG
일 NNG
친구 NNG
가슴 NNG
생각 NNG
손 NNG
때 NNG
사람 NNG
가슴 NNG
생각 NNG
손 NNG
때 NNG
사람 NNG
때 NNG
첫사랑 NNP
진행 NNG


# 람다

In [47]:
def add(a, b):
    return a + b

(lambda c, d : c + d)(1, 2)

3

# selenium

In [4]:
from selenium import webdriver
driver = webdriver.Chrome("chromedriver.exe")
driver.get("http://www.naver.com")